# <center> <u>**Premier réseau de classification : Resnet 50 , simple** </u></center>



# 1. Import des données


In [2]:
import os 
import pandas as pd
import numpy as np
import keras
import tensorflow as tf
from sklearn.datasets import load_files

2023-07-11 02:59:58.567263: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-07-11 02:59:58.807030: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [3]:
# main folder
path_to_folder = '../data_bees_detection/dataset_classification'

classes = os.listdir(os.path.join(path_to_folder, 'train'))
nb_classes = len(classes)  

# function to load a split of the dataset
def load_dataset(path):
    data = load_files(path)
    target_files = np.array(data['filenames'])
    target_labels = keras.utils.to_categorical(np.array(data['target']), nb_classes)
    return target_files, target_labels

# load the datasets
train_files, train_labels = load_dataset(os.path.join(path_to_folder, 'train'))
valid_files, valid_labels = load_dataset(os.path.join(path_to_folder, 'validation'))
test_files, test_labels = load_dataset(os.path.join(path_to_folder, 'test'))

In [19]:
# Batch generator

from keras.preprocessing import image
from keras.applications.resnet import preprocess_input
from keras.utils import Sequence
from keras.utils import load_img, img_to_array



def path_to_tensor(img_path):
    # loads RGB image as PIL.Image.Image type
    img = image.load_img(img_path, target_size=(224, 224))
    # convert PIL.Image.Image type to 3D tensor with shape (224, 224, 3)
    x = image.img_to_array(img)
    # convert 3D tensor to 4D tensor with shape (1, 224, 224, 3) and return 4D tensor
    return preprocess_input(np.expand_dims(x, axis=0))

class BatchGenerator(Sequence):

    def __init__(self, paths, labels, batch_size, augmentations):
        self.paths = paths
        self.labels = labels
        self.batch_size = batch_size
        self.augment = augmentations

    def __len__(self):
        # nb of batches per epoch
        return int(np.ceil(len(self.paths) / float(self.batch_size)))
    
    def load_img(self, img_path):

        # loads img 
        img = load_img(img_path, target_size=(224,224))
        x = img_to_array(img)
        
        # convert 3D tensor to 4D tensor with shape (1, 224, 224, 3) and return 4D tensor
        x =  np.expand_dims(x, axis=0)
        x = preprocess_input(x)
        return x


    def __getitem__(self, idx):
        # get batch at position index

        batch_paths = self.paths[idx * self.batch_size:(idx + 1) * self.batch_size]
        batch_labels = self.labels[idx * self.batch_size:(idx + 1) * self.batch_size]

        batch_imgs = np.zeros((len(batch_paths), 224, 224, 3))
        for i, f in enumerate(batch_paths):
            batch_imgs[i] = self.load_img(f)

        return batch_imgs, batch_labels




In [5]:
ds_train = BatchGenerator(train_files, train_labels, 16, None)
ds_valid = BatchGenerator(valid_files, valid_labels, 16, None)
ds_test = BatchGenerator(test_files, test_labels, 16, None)

# 2. Choix de l'architecture du modèle, des paramètres et hyperparamètres

In [6]:
from keras.applications.resnet import ResNet50
from keras.models import Model
from keras.layers import Dense, GlobalAveragePooling2D, Dropout

# we take resnet50 as convolutional base with weights trained on imagenet
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# we add the classification layers
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
x = Dropout(0.5)(x)
predictions = Dense(nb_classes, activation='softmax')(x)

# we create the final model
model = Model(inputs=base_model.input, outputs=predictions)


2023-07-11 03:00:35.294682: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-07-11 03:00:35.310724: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-07-11 03:00:35.310979: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

In [8]:
from keras.optimizers import Adam

opti = Adam(learning_rate=0.0001)
metrics = [tf.keras.metrics.CategoricalAccuracy(), tf.keras.metrics.Precision(), tf.keras.metrics.Recall()]
model.compile(optimizer=opti, loss='categorical_crossentropy', metrics=metrics)


In [7]:
# add callbacks
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau

# checkpoint to save the best model
path_to_weights = '/datafiles/classification/saved_weights/1.simple_resnet.h5'
checkpoint = ModelCheckpoint(filepath='best_model.hdf5', monitor='val_categorical_accuracy', save_best_only=True, save_weights_only=False)

# early stopping
early_stopping = EarlyStopping(monitor='val_categorical_accuracy', patience=10)

# reduce learning rate
reduce_lr = ReduceLROnPlateau(monitor='val_categorical_accuracy', factor=0.1, patience=5, min_lr=0.00001)

# 3. Entrainement du modèle

NB : poids saved in proper place

In [8]:
history = model.fit_generator(ds_train, epochs=100, validation_data=ds_valid, callbacks=[checkpoint, early_stopping, reduce_lr])

Epoch 1/100


/tmp/ipykernel_5648/3742308348.py:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  history = model.fit_generator(ds_train, epochs=100, validation_data=ds_valid, callbacks=[checkpoint, early_stopping, reduce_lr])
2023-06-30 22:41:36.194379: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]
2023-06-30 22:41:40.552601: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:424] Loaded cuDNN version 8600
2023-06-30 22:41:42.535615: W tensorflow/tsl/framework/bfc_allocator.cc:296] Allocator (GPU_0_bfc) ran out of memory trying to allocate 1.14GiB with freed_by_count=0. The caller indicates that this is not a failure, but this may mean that there co

3716/3716 [==============================] - ETA: 0s - loss: 3.2534 - categorical_accuracy: 0.2830 - precision: 0.6591 - recall: 0.1295

2023-06-30 23:04:15.199673: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]


3716/3716 [==============================] - 1471s 394ms/step - loss: 3.2534 - categorical_accuracy: 0.2830 - precision: 0.6591 - recall: 0.1295 - val_loss: 2.1893 - val_categorical_accuracy: 0.4693 - val_precision: 0.7355 - val_recall: 0.3088 - lr: 1.0000e-04
Epoch 2/100
3716/3716 [==============================] - 1450s 390ms/step - loss: 1.8882 - categorical_accuracy: 0.5212 - precision: 0.7671 - recall: 0.3734 - val_loss: 1.7139 - val_categorical_accuracy: 0.5656 - val_precision: 0.7638 - val_recall: 0.4556 - lr: 1.0000e-04
Epoch 3/100
3716/3716 [==============================] - 1443s 388ms/step - loss: 1.3029 - categorical_accuracy: 0.6478 - precision: 0.8244 - recall: 0.5377 - val_loss: 1.6464 - val_categorical_accuracy: 0.5810 - val_precision: 0.7565 - val_recall: 0.4897 - lr: 1.0000e-04
Epoch 4/100
3716/3716 [==============================] - 1443s 388ms/step - loss: 0.9373 - categorical_accuracy: 0.7373 - precision: 0.8610 - recall: 0.6537 - val_loss: 1.5072 - val_categorical

KeyboardInterrupt: 

: 

# 4. Test

In [22]:
import keras

path_test = '../data_bees_detection/dataset_classification/test'

IMG_SIZE = 224
train_ds = keras.utils.image_dataset_from_directory(
    directory=path_test,
    labels='inferred',
    label_mode='categorical',
    shuffle = False,
    batch_size=16,
    image_size=(IMG_SIZE, IMG_SIZE))


class_names = train_ds.class_names
nb_classes = len(class_names)


X,y = load_dataset(path_test)

X = path_to_tensor(X)


Found 19117 files belonging to 258 classes.


AttributeError: module 'keras.preprocessing.image' has no attribute 'load_img'

In [18]:
path_to_weights = '../../datafiles/classification/saved_weights/benchmark_weights/1_simple_resnet.hdf5'
model.load_weights(path_to_weights)

